<a href="https://colab.research.google.com/github/VighneshAlevoor/DeepLearning/blob/master/ANN_use_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv("Churn_Modelling.csv")
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
x=data.iloc[:,3:-1].values
y=data.iloc[:,-1:].values
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [5]:
label1=LabelEncoder()
x[:,1]=label1.fit_transform(x[:,1])
label2=LabelEncoder()
x[:,2]=label2.fit_transform(x[:,2])
x

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [6]:
colT=ColumnTransformer([('Encoding',OneHotEncoder(),[1])], remainder='passthrough')
x=colT.fit_transform(x)
x

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [7]:
x=x[:,1:]
x.shape

(10000, 11)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [0]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dropout
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [17]:
def build_grid(optimizer):
  classifier=Sequential()
  classifier.add(Dense(output_dim=6,activation='relu',input_dim=11, init='uniform'))
  classifier.add(Dropout(rate=0.1))
  classifier.add(Dense(output_dim=6,activation='relu', init='uniform'))
  classifier.add(Dropout(rate=0.1))
  classifier.add(Dense(output_dim=1,activation='sigmoid', init='uniform'))
  classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  return classifier
classifier=KerasClassifier(build_fn=build_grid)
param={'batch_size':[25,32], 'nb_epoch': [100,500], 'optimizer':['adam','rmsprop']}
gridSearch=GridSearchCV(estimator=classifier, param_grid=param,scoring='accuracy', cv=10)
gridSearch=gridSearch.fit(x_train,y_train)
print("Best parameters:   ", gridSearch.best_params_)
print("Best score :   ", gridSearch.best_score_)

Best parameters:    {'batch_size': 25, 'nb_epoch': 100, 'optimizer': 'adam'}
Best score :    0.796
